In [0]:
# Accessing Azure data lake storage gen2 with Azure databricks

In [0]:
spark.conf.set(
    "fs.azure.account.key.yogaaastorage.dfs.core.windows.net",'CT/fTJDVG4sbPGPGvLYeAWErDdwwtfMSEgzfLSwrAF+A8EIhHG1xqhwccElx6c97AmPIm4bbECcw+ASt9opWkw==')

In [0]:
# spark.read.load("abfss://retaildata@yogaaastorage.dfs.core.windows.net/<path-to-data>")

dbutils.fs.ls("abfss://retaildata@yogaaastorage.dfs.core.windows.net")

[FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1764696337000),
 FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1764696350000),
 FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1764696344000)]

In [0]:
dbutils.fs.ls("abfss://retaildata@yogaaastorage.dfs.core.windows.net/")

[FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1764696337000),
 FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1764696350000),
 FileInfo(path='abfss://retaildata@yogaaastorage.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1764696344000)]

In [0]:
df_product = spark.read.parquet("abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/product/"
)
df_store=spark.read.parquet("abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/store/"
)
df_transaction=spark.read.parquet("abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/transaction")
df_customer=spark.read.parquet("abfss://retaildata@yogaaastorage.dfs.core.windows.net/bronze/customer"
                            )



In [0]:
df_product.display()

product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
2,Bluetooth Speaker,Electronics,1299.49
3,Yoga Mat,Fitness,499.00
4,Laptop Stand,Accessories,999.99
5,Notebook Set,Stationery,149.00
6,Water Bottle,Fitness,299.00
7,Smartwatch,Electronics,4999.00
8,Desk Organizer,Accessories,399.00
9,Dumbbell Set,Fitness,1999.00
10,Pen Drive 32GB,Electronics,599.00


In [0]:
# Checking the data types of transcation 
df_transaction.dtypes

[('transaction_id', 'int'),
 ('customer_id', 'int'),
 ('product_id', 'int'),
 ('store_id', 'int'),
 ('quantity', 'int'),
 ('transaction_date', 'date')]

In [0]:
# Checking the data types of store
df_store.dtypes


[('store_id', 'int'), ('store_name', 'string'), ('location', 'string')]

In [0]:
# Checking the datatypes of product datatyoes
df_product.dtypes


[('product_id', 'int'),
 ('product_name', 'string'),
 ('category', 'string'),
 ('price', 'decimal(10,2)')]

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
display(df_store)

store_id,store_name,location
1,City Mall Store,Mumbai
2,High Street Store,Delhi
3,Tech World Outlet,Bangalore
4,Downtown Mini Store,Pune
5,Mega Plaza,Chennai


In [0]:
display(df_customer)

customer_id,first_name,last_name,email,phone,city,registration_date
101,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14
102,Nina,Joshi,user102@example.com,9876543210,Mumbai,2024-01-21
103,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10
104,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05
105,Riya,Singh,user105@example.com,9843210987,Chennai,2023-06-28
106,Ajay,Mishra,user106@example.com,9832109876,Pune,2024-03-10
107,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12
108,Rahul,Verma,user108@example.com,9810987654,Kolkata,2023-08-19
109,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01
110,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14


In [0]:
df_customer.dtypes

[('customer_id', 'bigint'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone', 'string'),
 ('city', 'string'),
 ('registration_date', 'string')]

In [0]:
df_customer=df_customer.dropDuplicates()

In [0]:
display(df_customer)

customer_id,first_name,last_name,email,phone,city,registration_date
112,Manoj,Sharma,user112@example.com,9776543210,Hyderabad,2023-09-30
128,Simran,Joshi,user128@example.com,9610987654,Hyderabad,2023-06-11
139,Harsha,Patel,user139@example.com,9509876543,Ahmedabad,2024-03-08
134,Jaya,Yadav,user134@example.com,9554321098,Mumbai,2023-08-08
144,Abhay,Yadav,user144@example.com,9454321098,Hyderabad,2024-01-06
127,Rohit,Mehta,user127@example.com,9621098765,Bangalore,2023-08-30
145,Ritu,Kapoor,user145@example.com,9443210987,Chennai,2023-11-09
101,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14
123,Arjun,Yadav,user123@example.com,9665432109,Ahmedabad,2024-02-25
136,Nilesh,Kapoor,user136@example.com,9532109876,Hyderabad,2024-01-30


In [0]:
# Creating a silver layer by joining all data

In [0]:
df_silver=df_transaction.join(df_customer,"customer_id").join(df_product,"product_id").join(df_store,"store_id").withColumn("Total_amount",col("quantity")*col("price"))

In [0]:
df_silver.display()

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,Total_amount
1,8,127,27,1,2024-10-10,Rohit,Mehta,user127@example.com,9621098765,Bangalore,2023-08-30,Desk Organizer,Accessories,399.00,City Mall Store,Mumbai,399.00
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14,Smartwatch,Electronics,4999.00,Mega Plaza,Chennai,14997.00
3,1,123,9,2,2024-10-08,Arjun,Yadav,user123@example.com,9665432109,Ahmedabad,2024-02-25,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01,Desk Organizer,Accessories,399.00,Mega Plaza,Chennai,1995.00
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,9632109876,Mumbai,2023-07-17,Water Bottle,Fitness,299.00,High Street Store,Delhi,598.00
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,9598765432,Pune,2023-12-22,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,298.00
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,9698765432,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,9,119,29,2,2025-06-03,Kunal,Nair,user119@example.com,9709876543,Bangalore,2023-08-10,Dumbbell Set,Fitness,1999.00,Downtown Mini Store,Pune,3998.00
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.00,Tech World Outlet,Bangalore,1996.00
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,9676543210,Pune,2023-09-20,Dumbbell Set,Fitness,1999.00,Downtown Mini Store,Pune,3998.00


In [0]:
# Dumping into the Adls storage

In [0]:
silver_path = "abfss://retaildata@yogaaastorage.dfs.core.windows.net/silver/"


In [0]:
display(silver_path)

'abfss://retaildata@yogaaastorage.dfs.core.windows.net/silver/'

In [0]:
df_silver.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
df_silver_read = spark.read.format("delta").load(silver_path)

In [0]:
display(df_silver_read)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,Total_amount
1,8,127,27,1,2024-10-10,Rohit,Mehta,user127@example.com,9621098765,Bangalore,2023-08-30,Desk Organizer,Accessories,399.00,City Mall Store,Mumbai,399.00
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,9887654321,Delhi,2023-09-14,Smartwatch,Electronics,4999.00,Mega Plaza,Chennai,14997.00
3,1,123,9,2,2024-10-08,Arjun,Yadav,user123@example.com,9665432109,Ahmedabad,2024-02-25,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,9809876543,Delhi,2024-04-01,Desk Organizer,Accessories,399.00,Mega Plaza,Chennai,1995.00
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,9632109876,Mumbai,2023-07-17,Water Bottle,Fitness,299.00,High Street Store,Delhi,598.00
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,9598765432,Pune,2023-12-22,Notebook Set,Stationery,149.00,City Mall Store,Mumbai,298.00
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,9698765432,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95
4,9,119,29,2,2025-06-03,Kunal,Nair,user119@example.com,9709876543,Bangalore,2023-08-10,Dumbbell Set,Fitness,1999.00,Downtown Mini Store,Pune,3998.00
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.00,Tech World Outlet,Bangalore,1996.00
4,9,122,23,2,2025-04-30,Tina,Kapoor,user122@example.com,9676543210,Pune,2023-09-20,Dumbbell Set,Fitness,1999.00,Downtown Mini Store,Pune,3998.00


In [0]:
df_silver_read.dtypes

[('store_id', 'int'),
 ('product_id', 'int'),
 ('customer_id', 'int'),
 ('transaction_id', 'int'),
 ('quantity', 'int'),
 ('transaction_date', 'date'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('email', 'string'),
 ('phone', 'string'),
 ('city', 'string'),
 ('registration_date', 'string'),
 ('product_name', 'string'),
 ('category', 'string'),
 ('price', 'decimal(10,2)'),
 ('store_name', 'string'),
 ('location', 'string'),
 ('Total_amount', 'decimal(21,2)')]

In [0]:
gold_df = df_silver_read.groupBy(
    "transaction_date",
    "product_id", "product_name", "category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)

In [0]:
display(gold_df)

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.00,1,399.000000
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.980000
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.00,1,1995.000000
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.00,1,1996.000000
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.00,1,596.000000
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.990000
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.950000
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.97,1,2399.970000
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.00,1,3998.000000
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.450000


In [0]:
# Dumping the data into gold container

In [0]:
gold_path = "abfss://retaildata@yogaaastorage.dfs.core.windows.net/gold/"

In [0]:
display(gold_path)
gold_df.write.mode("overwrite").format("delta").save(gold_path)

'abfss://retaildata@yogaaastorage.dfs.core.windows.net/gold/'

In [0]:

df_gold_read = spark.read.format("delta").load(gold_path)

In [0]:
display(df_gold_read)

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.00,1,399.000000
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.980000
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.00,1,1995.000000
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.00,1,1996.000000
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.00,1,596.000000
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.990000
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.950000
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.97,1,2399.970000
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.00,1,3998.000000
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.450000
